In [ ]:
import requests
from IPython.display import HTML
import csv
import dateutil.parser
from newspaper import Article
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
heading=["DatePublished","Name","Text","Summary","Keywords","Label","DateLimit"]
with open('bingACLEDNewsMatch.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(heading)  

In [ ]:
df = pandas.read_csv('ACLED2019data.csv')
training_data =[]
validation_data = []
acled_Data = []
start_date = dateutil.parser.parse('01-Jan-19')
end_date= dateutil.parser.parse('28-Feb-19')
for index, row in df.iterrows():
    date_pub = dateutil.parser.parse(row['event_date'])
    if start_date <= date_pub <= end_date:
        arr = [row['data_id'],row['event_date'],row['event_type'],row['region'],row['country'],row['location'],row['latitude'],row['longitude'],row['notes']]
        training_data.append(arr)
    else:
        arr = [row['data_id'],row['event_date'],row['event_type'],row['region'],row['country'],row['location'],row['latitude'],row['longitude'],row['notes']]
        validation_data.append(arr)
    acled_Data.append(arr)


In [ ]:
def cosine_Similarity(data):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data)
    cosine_mat = (X * X.T).A
    return cosine_mat

In [ ]:
def diff_dates(date1, date2):
    return abs(date2-date1).days

In [ ]:
# for each row it prints the index of similar set of records 
def getAcledSimilarityIndex(cosine_mat,news_date):
    similar_matrix = []
    similar_index = []
    isTrue = 0
    
    #this will filter the news for a cosine similarity of more than 0.3
    for i in range(len(cosine_mat[0])):
        if cosine_mat[0][i] >= 0.30:
            similar_index.append(i)
    
    if len(similar_index) > 0:
        isTrue = 1
        
    for j in range(len(similar_index)):  
        acled_date = dateutil.parser.parse(np.array(acled_Data)[similar_index[j]-1,1])
        if(news_date <= acled_date):
            similar_matrix.append(diff_dates(news_date,acled_date))
    
    return isTrue,similar_matrix

In [ ]:
new_df = pandas.read_csv('getBingNewsProcessed.csv')
News_training_data = []
start_date = dateutil.parser.parse('01-Jan-19')
end_date= dateutil.parser.parse('28-Feb-19')
count=0
for index, row in new_df.iterrows():
    print(count)
    match_news_and_acled = np.array(acled_Data)[:,8]
    match_news_and_acled = np.insert(match_news_and_acled, 0, row['Text'], axis=0)
    date_pub = dateutil.parser.parse(row['DatePublished'])
    mat = cosine_Similarity(match_news_and_acled)
    isCausedProtest,acled_matched_Dates = getAcledSimilarityIndex(mat,date_pub)
    if len(acled_matched_Dates) > 0:
        dateDiff = max(acled_matched_Dates)
    else:
        dateDiff = 0        
    arr = [date_pub,row['Name'],row['Text'],row['Summary'],row['Keywords'],isCausedProtest,dateDiff]
    News_training_data.append(arr)
    count=count+1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
for i in range(len(News_training_data)):
    with open('bingACLEDNewsMatch.csv', 'a') as csvFile:
        arr = [News_training_data[i][0].isoformat(),News_training_data[i][1],News_training_data[i][2],News_training_data[i][3],News_training_data[i][4],News_training_data[i][5],News_training_data[i][6]]
        writer = csv.writer(csvFile)
        writer.writerow(arr)  
    